In [ ]:
### Crawling my own melon playlist

In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd

In [2]:
seq = "PLAYLIST_NUMBER"
url = "https://www.melon.com/mymusic/playlist/mymusicplaylistview_listSong.htm?plylstSeq={}".format(seq)
headers = {"User-Agent" : UserAgent().chrome}
response = requests.get(url, headers=headers)
dom = BeautifulSoup(response.text, 'html.parser')

In [3]:
response

<Response [200]>

In [4]:
elements = dom.select("#frm > div > table > tbody tr")
len(elements)

50

In [5]:
element = elements[0]

In [6]:
artist = element.select_one("div.ellipsis > a.fc_mgray").text.replace('\n', '')
title = element.select_one("div.ellipsis > a.fc_gray").text
link = 'https://www.melon.com/song/detail.htm?songId=' + element.select_one('td > div > input').get('value')

artist, title, print(link)

https://www.melon.com/song/detail.htm?songId=732516


('버즈', '나에게로 떠나는 여행', None)

In [7]:
datas = []
for element in elements:
    data = {
        "Title" : element.select_one("a.fc_gray").text,
        "Artist" : element.select_one("#artistName").text.replace('\n', '')
    }
    datas.append(data)
datas

[{'Title': '나에게로 떠나는 여행', 'Artist': '버즈'},
 {'Title': '나비소녀 (Don`t Go)', 'Artist': 'EXO'},
 {'Title': '피터팬 (Peter Pan)', 'Artist': 'EXO'},
 {'Title': 'Baby, Don`t Cry (인어의 눈물)', 'Artist': 'EXO'},
 {'Title': '코끝에 겨울', 'Artist': '어반자카파'},
 {'Title': '거꾸로 걷는다', 'Artist': '어반자카파'},
 {'Title': '서울 밤 (feat. 빈지노)', 'Artist': '어반자카파'},
 {'Title': '그때의 나, 그때의 우리', 'Artist': '어반자카파'},
 {'Title': 'Boogie On & On', 'Artist': '빈지노 (Beenzino)'},
 {'Title': '목요일 밤 (Feat. 빈지노)', 'Artist': '어반자카파'},
 {'Title': '나빠(NAPPA)', 'Artist': 'Crush'},
 {'Title': 'Tip Toe (with 이하이)', 'Artist': 'Crush'},
 {'Title': '감아 (Feat. Crush)', 'Artist': '로꼬'},
 {'Title': 'Madeleine Love', 'Artist': 'CHEEZE (치즈)'},
 {'Title': '어떻게 생각해', 'Artist': 'CHEEZE (치즈)'},
 {'Title': 'Mood Indigo', 'Artist': 'CHEEZE (치즈)'},
 {'Title': '영화 같던 날', 'Artist': 'CHEEZE (치즈)'},
 {'Title': '연애 같은 걸 하니까', 'Artist': '소란 (SORAN)'},
 {'Title': '나만 알고 싶다', 'Artist': '소란 (SORAN)'},
 {'Title': '가을목이', 'Artist': '소란 (SORAN)'},
 {'Title': 'Perfect D

In [8]:
# 유저 플레이리스트 수집 함수

In [9]:
def playlist(seq):

        url = "https://www.melon.com/mymusic/playlist/mymusicplaylistview_listSong.htm?plylstSeq={}".format(seq)
        headers = {
            "User-Agent" : UserAgent().chrome
        }
        response = requests.get(url, headers=headers)
        dom = BeautifulSoup(response.text, 'html.parser')
        elements = dom.select("#frm > div > table > tbody tr")
        
        datas = []
        for element in elements:
            data = {
                "Title" : element.select_one("a.fc_gray").text,
                "Artist" : element.select_one("#artistName").text.replace('\n', ''),
                "Link" : 'https://www.melon.com/song/detail.htm?songId=' + element.select_one('td > div > input').get('value')
                }
            datas.append(data)
    
        return pd.DataFrame(datas)

In [22]:
link = yj_list.loc[0, 'Link']
print(link)

https://www.melon.com/song/detail.htm?songId=732516


In [11]:
headers = {"User-Agent" : UserAgent().chrome}
response = requests.get(link, headers=headers)
dom = BeautifulSoup(response.content, 'html.parser')
response

<Response [200]>

In [12]:
elements = dom.select_one('dd:nth-of-type(3)').text
elements

'록/메탈'

In [13]:
# 유저 플레이리스트의 장르 수집 함수

In [17]:
def playlist_genre(link):
    headers = {"User-Agent" : UserAgent().chrome}
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.content, 'html.parser')
    
    return dom.select_one('dd:nth-of-type(3)').text

In [18]:
def get_lyrics(link):
    headers = {"User-Agent" : UserAgent().chrome}
    response = requests.get(link, headers=headers)
    dom = BeautifulSoup(response.content, 'html.parser')
    elements = dom.select_one('.wrap_lyric').text.replace('\r', '').replace('\n', '').replace('\t', '').split('펼치기')[0]
    if elements == '[가사 준비중] 멜론 회원 여러분! 가사 등록을 기다리고 있어요.가사등록하기':
        elements = '-'
        
    return elements

In [19]:
yj_list = playlist(seq)

In [20]:
yj_list['Genre'] = yj_list['Link'].apply(playlist_genre)
yj_list.tail()

,Title,Artist,Link,Genre
45,잊는다는 게,윤현상,https://www.melon.com/song/detail.htm?songId=5...,발라드
46,오래된 노래,스탠딩 에그,https://www.melon.com/song/detail.htm?songId=3...,"인디음악, 포크/블루스"
47,꽃 (Feat. 타블로 Of 에픽하이),XIA (준수),https://www.melon.com/song/detail.htm?songId=5...,R&B;/Soul
48,Little Star,스탠딩 에그,https://www.melon.com/song/detail.htm?songId=3...,"인디음악, 포크/블루스"
49,홀로 (Feat. 김나영),정키,https://www.melon.com/song/detail.htm?songId=3...,발라드


In [21]:
yj_list['Lyrics'] = yj_list['Link'].apply(get_lyrics)
yj_list.tail()

,Title,Artist,Link,Genre,Lyrics
45,잊는다는 게,윤현상,https://www.melon.com/song/detail.htm?songId=5...,발라드,잊는다는 게 참 그래요사랑하다가 이별하게 되면그때서야 못해줬던 게 생각이 나요꼭 어...
46,오래된 노래,스탠딩 에그,https://www.melon.com/song/detail.htm?songId=3...,"인디음악, 포크/블루스",오래 전에 함께 듣던 노래가발걸음을 다시 멈춰서게 해이 거리에서너를 느낄 수 있어널...
47,꽃 (Feat. 타블로 Of 에픽하이),XIA (준수),https://www.melon.com/song/detail.htm?songId=5...,R&B;/Soul,Take my handYou know I’m here with youSay my n...
48,Little Star,스탠딩 에그,https://www.melon.com/song/detail.htm?songId=3...,"인디음악, 포크/블루스",눈을 감고내가 하는 이야기를 잘 들어봐나의 얘기가 끝나기 전에너는 꿈을 꿀거야Lit...
49,홀로 (Feat. 김나영),정키,https://www.melon.com/song/detail.htm?songId=3...,발라드,아침에 눈을 떴을 때텅 빈 방안에 나 홀로 니 빈자리 거닐면서많은 생각들에 잠겨지워...


In [24]:
yj_list.to_csv('C:/Users/luvu1/Documents/dss15/crawling_project/datas/yj_list.csv')